In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import urllib.request
from PIL import Image


In [9]:
def get_number_rating(driver):
    try:
        return(driver.find_element(By.XPATH,"/html/body/div[4]/div/div/section[1]/div/div/div[1]/div[2]/div[1]/div/div/span").text)
    except:
        return(None)

def get_word_rating(driver):
    try:
        return(driver.find_element(By.XPATH,"/html/body/div[4]/div/div/section[1]/div/div/div[1]/div[2]/div[2]/div[1]").text)
    except:
        return(None)

def get_num_rating(driver):
    try:
        return(driver.find_element(By.XPATH,"/html/body/div[4]/div/div/section[1]/div/div/div[1]/div[2]/div[2]/div[2]").text)
    except:
        return(None)
    
def get_single_info(driver):
    #Pull list, Collection, Read, Wishlist
    temp_list = []
    number_block = driver.find_elements(By.CLASS_NAME,"ml-1")
    for i in range(0,4):
        temp_list.append(number_block[i].text)
    if '@' in temp_list[-1]: #for older comics that do not have pulls numbers anymore
        temp_list.pop(-1)
        temp_list.insert(0,None)
    return temp_list

def get_name(driver):
    return(driver.find_element(By.XPATH,"/html/body/div[4]/div/div/header/section[1]/div[3]/div/div/h1").text)

def get_date(driver):
    return(driver.find_element(By.XPATH,'/html/body/div[4]/div/div/header/section[1]/div[3]/div/div/div/a[2]').text)

def get_img(driver,name):
    name = name.replace(' ','_')
    name = name.replace('#','').replace(':','').lower()
    img_src = driver.find_element(By.XPATH,'/html/body/div[4]/div/div/section[2]/div/div[1]/div/aside/div/div[1]/div[1]/a/img')
    photo_url = img_src.get_attribute('src')
    urllib.request.urlretrieve( 
       f'{photo_url}', 
       f"{name}.png") 
    img = Image.open(f"{name}.png")

In [11]:
def scrape_comics(url_list):
    '''Scrapes League of Comic Geeks. url_list should contain urls on comic book series pages 
    such as https://leagueofcomicgeeks.com/comics/series/168170/transformers'''
    
    comic_info_list = []
    for url in url_list:
        comic_url_list = []
        options = webdriver.ChromeOptions()
        options.add_argument("--start-maximized")
        driver = webdriver.Chrome(options=options)
        
        driver.get(url)
        
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR,"div.title.color-primary > a"))
        )
        
        for comic in driver.find_elements(By.CSS_SELECTOR,"div.title.color-primary > a"):
            comic_url = comic.get_attribute('href')
            if 'variant' not in comic_url: #Only grabs cover A comics
                comic_url_list.append(comic_url)
                
        print(comic_url_list)
        
        for url in comic_url_list:
            driver.get(url)
            element = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.XPATH,"/html/body/div[4]/div/div/header/section[1]/div[3]/div/div/h1"))
            )
            
            name = get_name(driver)
            number_rating = get_number_rating(driver)
            word_rating = get_word_rating(driver)
            num_rating = get_num_rating(driver)
            date = get_date(driver)
            stat_list = get_single_info(driver)
            get_img(driver,name)
            
            comic_info_list.append([name,date,number_rating,word_rating,num_rating]+stat_list)
            print([name,date,number_rating,word_rating,num_rating]+stat_list)
            
        driver.quit()
    return(comic_info_list)